In [18]:
import os
from pyannote.core import Segment
from dotenv import load_dotenv
from pyannote.audio import Pipeline

In [3]:
load_dotenv()
token = os.getenv("HF_TOKEN")

In [4]:
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=token)

C:\Users\aodih\AppData\Local\Programs\Python\Python312\Lib\inspect.py:1007: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\aodih\.cache\torch\pyannote\models--pyannote--segmentation\snapshots\c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b\pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cpu. Bad things might happen unless you revert torch to 1.x.


C:\workspace\healper\whisper-re\.venv\Lib\site-packages\speechbrain\utils\fetching.py:151: UserWarning: Using SYMLINK strategy on Windows for fetching potentially requires elevated privileges and is not recommended. See `LocalStrategy` documentation.
  warnings.warn(
C:\workspace\healper\whisper-re\.venv\Lib\site-packages\speechbrain\utils\autocast.py:188: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)
C:\workspace\healper\whisper-re\.venv\Lib\site-packages\speechbrain\utils\parameter_transfer.py:234: UserWarning: Requested Pretrainer collection using symlinks on Windows. This might not work; see `LocalStrategy` documentation. Consider unsetting `collect_in` in Pretrainer to avoid symlinking altogether.
  warnings.warn(


In [6]:
diarization = pipeline("audio/example.mp3")

C:\workspace\healper\whisper-re\.venv\Lib\site-packages\torchaudio\_backend\soundfile_backend.py:71: UserWarning: The MPEG_LAYER_III subtype is unknown to TorchAudio. As a result, the bits_per_sample attribute will be set to 0. If you are seeing this warning, please report by opening an issue on github (after checking for existing/closed ones). You may otherwise ignore this warning.
  warnings.warn(


In [8]:
res = []
for turn, _, speaker in diarization.itertracks(yield_label=True):
    res.append(f"Speaker {speaker}: [{turn.start:.1f}s - {turn.end:.1f}s]")

In [10]:
import whisper
model = whisper.load_model("base")  # You can also use "small", "medium", "large"
result = model.transcribe("audio/example.mp3", verbose=False)

C:\workspace\healper\whisper-re\.venv\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detected language: English


100%|██████████| 43065/43065 [00:28<00:00, 1498.13frames/s]


In [19]:
conv = []
for item in result["segments"]:
    start = item["start"]
    end = item["end"]
    text = item["text"]
    seg = Segment(start, end)
    speaker = diarization.crop(seg).argmax()
    conv.append((speaker, seg, text))


In [20]:
conv

[('SPEAKER_00', <Segment(0, 8.92)>, ' Listening activity number 11.'),
 ('SPEAKER_01',
  <Segment(8.92, 13.12)>,
  ' You will hear a conversation between a secretary and a student.'),
 ('SPEAKER_01',
  <Segment(13.12, 18.04)>,
  ' The secretary is asking the student for information in order to complete an application form for'),
 ('SPEAKER_01', <Segment(18.04, 19.76)>, ' a course.'),
 ('SPEAKER_01',
  <Segment(19.76, 25.8)>,
  ' As you listen, fill in the appropriate information on the form below.'),
 ('SPEAKER_01', <Segment(25.8, 26.8)>, ' Good afternoon.'),
 ('SPEAKER_01', <Segment(26.8, 27.8)>, ' Can I help you?'),
 ('SPEAKER_00',
  <Segment(28.04, 31.76)>,
  " Yes, I'd like to enroll for the English short training course."),
 ('SPEAKER_01', <Segment(31.76, 32.76)>, ' Certainly.'),
 ('SPEAKER_01', <Segment(32.76, 34.12)>, " What's your surname, please?"),
 ('SPEAKER_00', <Segment(34.12, 35.16)>, " Potter's."),
 ('SPEAKER_01', <Segment(35.16, 36.64)>, ' Could you spell that for me?')